In [1]:
import matplotlib.pyplot as plt
import numpy as np
import helper
import time
import datetime
import torch.nn as nn
import torchvision.models
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets, models
import torchvision.utils
import torch
import pandas as pd
from torchinfo import summary
from PIL import Image
from torchvision.transforms import ToTensor
from glob import glob
from torch.utils.data import Dataset, DataLoader, random_split
from copy import copy
from collections import defaultdict
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import time
from sklearn.metrics import classification_report
from tqdm import tqdm
import math
from torcheval.metrics import BinaryAccuracy
import os
import timm
import segmentation_models_pytorch as smp
import random
from sklearn.model_selection import train_test_split

device = torch.device("cuda:4" if torch.cuda.is_available() else "cpu")
batch_size=8
img_size=512
tf = ToTensor()
def expand2square(pil_img, background_color):
    width, height = pil_img.size
    if width == height:
        return pil_img
    elif width > height:
        result = Image.new(pil_img.mode, (width, width), background_color)
        result.paste(pil_img, (0, (width - height) // 2))
        return result
    else:
        result = Image.new(pil_img.mode, (height, height), background_color)
        result.paste(pil_img, ((height - width) // 2, 0))
        return result

In [2]:
img_path='../../data/area_segmentation/BRDC/image/'
img_list=glob(img_path+'*.jpeg')
mask_list=[i.replace('/image','/mask') for i in img_list]
mask_list=[i.replace('.jpeg','.npy') for i in mask_list]
X_train, X_test, Y_train, Y_test = train_test_split(img_list, mask_list, test_size=0.2, random_state=321)

test_image=torch.zeros((len(X_test),3,img_size,img_size))
test_mask=torch.zeros((len(Y_test),5,img_size,img_size),dtype=torch.uint8)    
train_image=torch.zeros((len(X_train),3,img_size,img_size))
train_mask=torch.zeros((len(Y_train),5,img_size,img_size),dtype=torch.uint8)

for i in tqdm(range(len(X_train))):
    train_image[i] = tf(np.array(Image.open(X_train[i]).resize((img_size, img_size))))
    train_mask[i]=tf(np.resize(np.load(Y_train[i]),(img_size,img_size,5)))
for i in tqdm(range(len(X_test))):
    test_image[i] = tf(np.array(Image.open(X_test[i]).resize((img_size, img_size))))
    test_mask[i]=tf(np.resize(np.load(Y_test[i]),(img_size,img_size,5)))

100%|██████████| 113/113 [00:21<00:00,  5.32it/s]


In [13]:
class CustomDataset(Dataset):
    def __init__(self, image_list, label_list):
        self.img_path = image_list
        self.label = label_list
        
    def trans(self,image,label):
        if random.random() > 0.5:
            transform = transforms.RandomHorizontalFlip(1)
            label = transform(label)
            image = transform(image)
            
        if random.random() > 0.5:
            transform = transforms.RandomVerticalFlip(1)
            label = transform(label)
            image = transform(image)
            
        return image,label
    
    def __len__(self):
        return len(self.img_path)

    def __getitem__(self, idx):
        image_path,label_path = self.trans(self.img_path[idx],self.label[idx])

        return image_path, label_path
    
train_dataset = CustomDataset(train_image, train_mask)

test_dataset = CustomDataset(test_image, test_mask)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

In [14]:
model=smp.UnetPlusPlus('efficientnet-b5', in_channels=3, classes=5).to(device)
summary(model,(batch_size,3,img_size,img_size))
def dice_loss(pred, target, num_classes=4):
    # Apply softmax to predictions
    pred_softmax = F.sigmoid(pred)
    
    # Ensure the target tensor is in float type
    target = target.float()
    dice_loss=torch.zeros(len(target))
    for i in range(len(target)):
        # Calculate the Dice coefficient for each class
        intersection = (pred_softmax[i] * target[i]).sum(dim=(1, 2))
        union = pred_softmax[i].sum(dim=(1,2)) + target[i].sum(dim=(1,2))
        
        dice_coefficient = 2. * intersection / (union + 1e-5)  # Adding a small epsilon to avoid division by zero
        
        # Calculate the Dice loss
        dice_loss[i] = 1 - dice_coefficient.mean()
    
    return dice_loss.mean()

In [11]:
train_loss_list=[]
val_loss_list=[]
train_acc_list=[]
val_acc_list=[]
MIN_loss=5000
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=2e-3)
metrics = defaultdict(float)
for epoch in range(1000):
    train=tqdm(train_dataloader)
    count=0
    running_loss = 0.0
    acc_loss=0
    for x, y in train:
        model.train()
        y = y.to(device).float()
        count+=1
        x=x.to(device).float()
        optimizer.zero_grad()  # optimizer zero 로 초기화
        predict = model(x).to(device)
        cost = dice_loss(predict, y) # cost 구함
        acc=1-cost.item()
        cost.backward() # cost에 대한 backward 구함
        optimizer.step() 
        running_loss += cost.item()
        acc_loss+=acc
        y = y.to('cpu')

        x=x.to('cpu')
        train.set_description(f"epoch: {epoch+1}/{1000} Step: {count+1} dice_loss : {running_loss/count:.4f} dice_score: {1-running_loss/count:.4f}")
    train_loss_list.append((running_loss/count))
    train_acc_list.append((acc_loss/count))
#test
    val=tqdm(test_dataloader)
    model.eval()
    count=0
    val_running_loss=0.0
    acc_loss=0
    with torch.no_grad():
        for x, y in val:
            y = y.to(device).float()
            count+=1
            x=x.to(device).float()
            
            predict = model(x).to(device)
            cost = dice_loss(predict, y) # cost 구함
            acc=1-cost.item()
            val_running_loss+=cost.item()
            acc_loss+=acc
            y = y.to('cpu')
            x=x.to('cpu')
            val.set_description(f"test epoch: {epoch+1}/{1000} Step: {count+1} dice_loss : {val_running_loss/count:.4f}  dice_score: {1-val_running_loss/count:.4f}")
        val_loss_list.append((val_running_loss/count))
        val_acc_list.append((acc_loss/count))
        
    if MIN_loss>(val_running_loss/count):
        torch.save(model.state_dict(), '../../model/areaSeg/UnetPlusPlus_callback.pt')
        MIN_loss=(val_running_loss/count)
        
    if epoch%50==5:
        plt.figure(figsize=(10,5))
        plt.subplot(1, 2, 1) 
        plt.title('loss_graph')
        plt.plot(np.arange(epoch+1),train_loss_list,label='train_loss')
        plt.plot(np.arange(epoch+1),val_loss_list,label='test_loss')
        plt.xlabel('epoch')
        plt.ylabel('loss')
        plt.ylim([0, 1]) 
        plt.legend()
        plt.subplot(1, 2, 2)  
        plt.title('acc_graph')
        plt.plot(np.arange(epoch+1),train_acc_list,label='train_acc')
        plt.plot(np.arange(epoch+1),val_acc_list,label='test_acc')
        plt.xlabel('epoch')
        plt.ylabel('accuracy')
        plt.ylim([0, 1]) 
        plt.legend()
        plt.show()
plt.figure(figsize=(10,5))
plt.subplot(1, 2, 1) 
plt.title('loss_graph')
plt.plot(np.arange(epoch+1),train_loss_list,label='train_loss')
plt.plot(np.arange(epoch+1),val_loss_list,label='test_loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.ylim([0, 1]) 
plt.legend()
plt.subplot(1, 2, 2)  
plt.title('acc_graph')
plt.plot(np.arange(epoch+1),train_acc_list,label='train_acc')
plt.plot(np.arange(epoch+1),val_acc_list,label='test_acc')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.ylim([0, 1]) 
plt.legend()
plt.show()
print('batch size= 2')
print('image size= 1024,1024')
print('learning rate= 0.0001')

epoch: 1/1000 Step: 57 dice_loss : 0.8192 dice_score: 0.1808: 100%|██████████| 56/56 [00:18<00:00,  3.03it/s]
test epoch: 1/1000 Step: 15 dice_loss : 0.7990  dice_score: 0.2010: 100%|██████████| 14/14 [00:01<00:00,  9.85it/s]
epoch: 2/1000 Step: 57 dice_loss : 0.8086 dice_score: 0.1914: 100%|██████████| 56/56 [00:18<00:00,  3.00it/s]
test epoch: 2/1000 Step: 15 dice_loss : 0.7981  dice_score: 0.2019: 100%|██████████| 14/14 [00:01<00:00,  8.98it/s]
epoch: 3/1000 Step: 13 dice_loss : 0.8056 dice_score: 0.1944:  21%|██▏       | 12/56 [00:04<00:15,  2.89it/s]


KeyboardInterrupt: 